In [ ]:
# # Install:
# # Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && git checkout 153397f9a4423884d3b469ad8b970367ffa45a22 && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.7 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.7.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *

import numpy
import math
import os
import numpy as np
import math
import cv2

distance_shot_threshold = 14
field_size = numpy.array([106,68])
wpc = 1
wepv = 0.3
wxg = 1
th_pass_d = 30 # meters
th_angle = 15 # degrees


#######################
# Utilities functions #
######################

def to_pixel_space(p, image_width, image_height):
    pxn = min(int(((p[0] + 1)/2)*image_width), image_width-1)
    pyn = min(int(((p[1] + 0.42)/0.84)*image_height), image_height-1)
    return pxn, pyn

def to_normalized_space(p):
    xn = (p[0] + 1)/2
    yn = (p[1] + 0.42)/0.84
    return xn, yn

def to_normalized_space_for_players(players):
    normalized_players = numpy.zeros_like(players)
    normalized_players[:,0] = (players[:, 0] + 1)/2
    normalized_players[:,1] = (players[:, 1] + 0.42)/0.84
    return normalized_players

def angle_between_vectors_degrees(v1, v2):
    
    try:
        dot_prod = numpy.dot(v1, v2)
        angle = math.acos(dot_prod)*180/math.pi  
    except:
        angle = -1000

    return angle

def get_epv_from_normalized_point(p, epv):

    p = p[0] + 0.5, -p[1] + 0.5

    if p[0] < 0 or p[0] >=1 or p[1]<0 or p[1] >=1:
        return 0
    
    pos_x = int(p[0]*epv.shape[1])
    pos_y = int(p[1]*epv.shape[0])

    epv_p = 0
    if 0 <= pos_x < epv.shape[1] and 0 <= pos_y < epv.shape[0]:
        epv_p = epv[pos_y, pos_x]

    return epv_p

def get_xg_from_normalized_point(p, epv):

    p = p[0] + 0.5, -p[1] + 0.5

    p =  min(1, max(0, p[0])), min(1, max(0, p[1]))
    
    pos_x = int(p[0]*epv.shape[1])
    pos_y = int(p[1]*epv.shape[0])

    epv_p = 0
    if 0 <= pos_x < epv.shape[1] and 0 <= pos_y < epv.shape[0]:
        epv_p = epv[pos_y, pos_x]

    return epv_p

# Return the direction must to orientate cotrolled player to pass to best player
def get_orientation_to_pass(controlled_player_pos_normalized,
                            best_player_pos_normalized):
  
    orientations_name = ["right", "topright", "top", "topleft", "left", "bottomleft", "bottom", "bottomright"]
    orientations = numpy.array([[1, 0], [0.707, 0.707], [0, 1], [-0.707, 0.707], [-1, 0], [-0.707, -0.707], [0, -1], [0.707, -0.707]]).reshape((-1,2))
    # orientations = numpy.array([[1, 0], [0.707, -0.707], [0, -1], [-0.707, -0.707], [-1, 0], [-0.707, 0.707], [0, 1], [0.707, 0.707]]).reshape((-1,2))

    pass_vector = best_player_pos_normalized - controlled_player_pos_normalized
    pass_vector_norm = pass_vector/numpy.linalg.norm(pass_vector)

    dot_prod = numpy.dot(orientations, pass_vector_norm.reshape((2,1)))

    idx = numpy.argmax(dot_prod.flatten())
    orientation = orientations_name[idx]
    if orientation == "right":
        return 5
    if orientation == "topright":
        return 4
    if orientation == "top":
        return 3
    if orientation == "topleft":
        return 2
    if orientation == "left":
        return 1
    if orientation == "bottomleft":
        return 8
    if orientation == "bottom":
        return 7
    if orientation == "bottomright":
        return 6

# Offside detection to avoid pass
def offside(controlled_player_pos_normalized, 
            best_player_pos_normalized,
            players_pos_right_normalized):
  
    th = 1/105
    pr = best_player_pos_normalized[0]
    pc = controlled_player_pos_normalized[0]
    right_x = numpy.max(players_pos_right_normalized[1:,0])

    # if player to recieve the ball is in its own field
    if best_player_pos_normalized[0] <= 0:
        return False

    if pr+th>right_x and pc < right_x:
        return True

    return False  

# Pass type selection
def get_pass_type(controlled_player_pos_normalized, controlled_player_dir_normalized, players_pos_left_normalized, players_pos_right_normalized):

    global field_size
    global th_pass_d
    global th_angle

    pass_dirs = players_pos_left_normalized - controlled_player_pos_normalized
    pass_dirs_norm = pass_dirs/numpy.linalg.norm(pass_dirs, axis=1).reshape((-1,1))
    deffender_dirs = players_pos_right_normalized - controlled_player_pos_normalized
    deffender_dirs_norm = deffender_dirs/numpy.linalg.norm(deffender_dirs, axis=1).reshape((-1,1))
    d = numpy.linalg.norm(pass_dirs*field_size, axis = 1)
    d_def = numpy.linalg.norm(deffender_dirs*field_size, axis = 1)
    pass_type = -numpy.ones(players_pos_left_normalized.shape[0], dtype=int)
    pass_type[d < th_pass_d] = 11
    pass_type[d >= th_pass_d] = 9
    angles_pass_and_orientation = []
    angles_pass_and_orientation_def = []

    for i, pdn in enumerate(pass_dirs_norm):
        angle_pass_and_orientation = angle_between_vectors_degrees(controlled_player_dir_normalized, pdn)
        angles_pass_and_orientation.append(angle_pass_and_orientation)
        for idx, ddn in enumerate(deffender_dirs_norm):
            try:
                angle = angle_between_vectors_degrees(pdn, ddn)
            except:
                angle = 1000
            if angle < th_angle and d_def[idx] < d[i]:
                pass_type[i] = 10
                break
                
    for idx, ddn in enumerate(deffender_dirs_norm):
        angle_pass_and_orientation = angle_between_vectors_degrees(controlled_player_dir_normalized, ddn)
        angles_pass_and_orientation_def.append(angle_pass_and_orientation)

    angles_pass_and_orientation = numpy.array(angles_pass_and_orientation)
    angles_pass_and_orientation_def = numpy.array(angles_pass_and_orientation_def)
    return pass_type, angles_pass_and_orientation, d, angles_pass_and_orientation_def, d_def

######################
# -------------------#
######################

#############
# EPV Model #
#############
# Extracted from: https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/master/EPV_grid.csv
def get_epv():
    epv = numpy.array([0.0046,0.0046,0.0045,0.0046,0.0048,0.0051,0.0054,0.0057,0.0059,0.0062,0.0066,0.0068,0.0071,0.0072,0.0073,0.0077,0.0082,0.0086,0.0088,0.0092,0.0096,0.0100,0.0104,0.0110,0.0117,0.0123,0.0127,0.0131,0.0135,0.0142,0.0150,0.0160,0.0170,0.0181,0.0192,0.0208,0.0227,0.0244,0.0259,0.0276,0.0297,0.0313,0.0326,0.0341,0.0356,0.0365,0.0368,0.0388,0.0424,0.0443,
    0.0044,0.0044,0.0045,0.0046,0.0049,0.0052,0.0055,0.0058,0.0060,0.0062,0.0066,0.0069,0.0072,0.0074,0.0075,0.0079,0.0084,0.0087,0.0090,0.0093,0.0097,0.0102,0.0106,0.0112,0.0118,0.0123,0.0128,0.0132,0.0136,0.0143,0.0152,0.0162,0.0171,0.0183,0.0196,0.0211,0.0229,0.0246,0.0261,0.0278,0.0298,0.0314,0.0327,0.0342,0.0359,0.0368,0.0370,0.0388,0.0421,0.0437,
    0.0041,0.0042,0.0044,0.0047,0.0052,0.0056,0.0058,0.0060,0.0061,0.0063,0.0067,0.0070,0.0074,0.0076,0.0079,0.0083,0.0087,0.0090,0.0092,0.0095,0.0101,0.0106,0.0111,0.0115,0.0119,0.0124,0.0130,0.0135,0.0139,0.0146,0.0156,0.0165,0.0174,0.0186,0.0202,0.0218,0.0232,0.0248,0.0265,0.0283,0.0300,0.0315,0.0328,0.0345,0.0366,0.0376,0.0376,0.0388,0.0413,0.0425,
    0.0041,0.0042,0.0045,0.0049,0.0054,0.0058,0.0061,0.0062,0.0062,0.0064,0.0068,0.0072,0.0075,0.0079,0.0082,0.0085,0.0089,0.0092,0.0094,0.0098,0.0104,0.0109,0.0114,0.0118,0.0122,0.0126,0.0133,0.0138,0.0142,0.0150,0.0160,0.0170,0.0179,0.0192,0.0209,0.0224,0.0237,0.0252,0.0268,0.0284,0.0300,0.0316,0.0332,0.0351,0.0373,0.0386,0.0392,0.0400,0.0411,0.0417,
    0.0044,0.0045,0.0048,0.0052,0.0056,0.0060,0.0062,0.0064,0.0065,0.0067,0.0070,0.0073,0.0077,0.0080,0.0083,0.0087,0.0092,0.0095,0.0097,0.0101,0.0107,0.0113,0.0117,0.0121,0.0125,0.0129,0.0136,0.0141,0.0146,0.0153,0.0164,0.0175,0.0186,0.0200,0.0215,0.0230,0.0243,0.0256,0.0269,0.0282,0.0296,0.0315,0.0339,0.0361,0.0380,0.0400,0.0418,0.0424,0.0416,0.0412,
    0.0045,0.0047,0.0051,0.0054,0.0057,0.0060,0.0063,0.0066,0.0067,0.0069,0.0072,0.0075,0.0079,0.0082,0.0085,0.0089,0.0094,0.0098,0.0100,0.0104,0.0111,0.0116,0.0120,0.0124,0.0127,0.0131,0.0138,0.0144,0.0149,0.0157,0.0169,0.0180,0.0191,0.0204,0.0219,0.0234,0.0248,0.0261,0.0274,0.0287,0.0300,0.0323,0.0357,0.0382,0.0399,0.0421,0.0447,0.0452,0.0438,0.0430,
    0.0047,0.0049,0.0053,0.0056,0.0057,0.0060,0.0064,0.0067,0.0068,0.0070,0.0075,0.0078,0.0080,0.0083,0.0087,0.0091,0.0096,0.0100,0.0103,0.0108,0.0115,0.0119,0.0123,0.0125,0.0127,0.0132,0.0140,0.0147,0.0153,0.0162,0.0174,0.0184,0.0192,0.0204,0.0221,0.0237,0.0251,0.0267,0.0285,0.0299,0.0311,0.0340,0.0387,0.0416,0.0429,0.0449,0.0476,0.0486,0.0477,0.0473,
    0.0047,0.0050,0.0054,0.0057,0.0059,0.0062,0.0067,0.0070,0.0070,0.0072,0.0077,0.0080,0.0082,0.0085,0.0090,0.0094,0.0098,0.0102,0.0106,0.0111,0.0117,0.0122,0.0125,0.0128,0.0131,0.0136,0.0143,0.0151,0.0157,0.0166,0.0177,0.0187,0.0195,0.0207,0.0223,0.0238,0.0253,0.0273,0.0299,0.0321,0.0341,0.0373,0.0417,0.0446,0.0460,0.0489,0.0534,0.0546,0.0525,0.0514,
    0.0048,0.0050,0.0055,0.0059,0.0062,0.0066,0.0071,0.0074,0.0074,0.0076,0.0079,0.0082,0.0085,0.0088,0.0092,0.0095,0.0100,0.0104,0.0107,0.0112,0.0119,0.0124,0.0128,0.0132,0.0136,0.0142,0.0148,0.0155,0.0163,0.0171,0.0179,0.0188,0.0199,0.0211,0.0225,0.0238,0.0252,0.0278,0.0316,0.0353,0.0390,0.0421,0.0447,0.0471,0.0492,0.0541,0.0619,0.0633,0.0582,0.0556,
    0.0050,0.0052,0.0058,0.0062,0.0065,0.0069,0.0074,0.0077,0.0077,0.0078,0.0081,0.0084,0.0088,0.0091,0.0094,0.0097,0.0102,0.0106,0.0109,0.0114,0.0120,0.0125,0.0131,0.0136,0.0141,0.0146,0.0152,0.0158,0.0166,0.0173,0.0180,0.0189,0.0201,0.0214,0.0227,0.0243,0.0263,0.0293,0.0333,0.0373,0.0413,0.0452,0.0490,0.0540,0.0600,0.0655,0.0703,0.0702,0.0651,0.0626,
    0.0053,0.0057,0.0063,0.0067,0.0068,0.0071,0.0076,0.0079,0.0079,0.0081,0.0083,0.0086,0.0089,0.0093,0.0096,0.0100,0.0103,0.0107,0.0113,0.0117,0.0121,0.0126,0.0132,0.0138,0.0145,0.0150,0.0155,0.0161,0.0168,0.0174,0.0180,0.0188,0.0200,0.0213,0.0229,0.0253,0.0286,0.0318,0.0349,0.0380,0.0410,0.0465,0.0546,0.0653,0.0786,0.0829,0.0784,0.0752,0.0734,0.0725,
    0.0060,0.0062,0.0067,0.0070,0.0071,0.0074,0.0078,0.0080,0.0081,0.0082,0.0084,0.0087,0.0091,0.0095,0.0098,0.0101,0.0105,0.0109,0.0115,0.0119,0.0123,0.0128,0.0134,0.0140,0.0146,0.0152,0.0156,0.0163,0.0171,0.0178,0.0183,0.0191,0.0201,0.0214,0.0230,0.0259,0.0300,0.0335,0.0362,0.0395,0.0432,0.0514,0.0639,0.0779,0.0934,0.0998,0.0974,0.0967,0.0980,0.0986,
    0.0069,0.0069,0.0069,0.0071,0.0074,0.0077,0.0080,0.0082,0.0081,0.0082,0.0085,0.0089,0.0093,0.0096,0.0099,0.0102,0.0106,0.0111,0.0116,0.0121,0.0125,0.0130,0.0136,0.0141,0.0145,0.0150,0.0156,0.0165,0.0177,0.0185,0.0189,0.0196,0.0205,0.0216,0.0230,0.0260,0.0306,0.0344,0.0373,0.0419,0.0481,0.0597,0.0769,0.0918,0.1045,0.1162,0.1271,0.1347,0.1390,0.1412,
    0.0077,0.0075,0.0072,0.0072,0.0076,0.0079,0.0082,0.0083,0.0081,0.0082,0.0086,0.0089,0.0093,0.0096,0.0099,0.0103,0.0107,0.0112,0.0116,0.0121,0.0126,0.0131,0.0137,0.0140,0.0142,0.0147,0.0156,0.0165,0.0175,0.0183,0.0190,0.0198,0.0208,0.0219,0.0232,0.0262,0.0310,0.0352,0.0386,0.0454,0.0555,0.0698,0.0882,0.1038,0.1165,0.1362,0.1628,0.1982,0.2425,0.2647,
    0.0082,0.0080,0.0075,0.0075,0.0078,0.0081,0.0084,0.0084,0.0082,0.0083,0.0085,0.0088,0.0092,0.0096,0.0099,0.0103,0.0108,0.0112,0.0115,0.0119,0.0125,0.0130,0.0136,0.0138,0.0136,0.0142,0.0156,0.0164,0.0166,0.0173,0.0184,0.0196,0.0210,0.0223,0.0235,0.0265,0.0313,0.0358,0.0400,0.0500,0.0656,0.0816,0.0979,0.1139,0.1296,0.1598,0.2044,0.2873,0.4085,0.4691,
    0.0085,0.0083,0.0077,0.0076,0.0079,0.0082,0.0085,0.0085,0.0083,0.0083,0.0085,0.0088,0.0092,0.0095,0.0099,0.0103,0.0108,0.0112,0.0114,0.0118,0.0124,0.0130,0.0136,0.0137,0.0133,0.0139,0.0156,0.0164,0.0162,0.0168,0.0181,0.0195,0.0211,0.0225,0.0237,0.0267,0.0315,0.0362,0.0408,0.0523,0.0707,0.0875,0.1027,0.1190,0.1362,0.1716,0.2252,0.3319,0.4915,0.5714,
    0.0085,0.0083,0.0077,0.0076,0.0079,0.0082,0.0085,0.0085,0.0083,0.0083,0.0085,0.0088,0.0092,0.0095,0.0099,0.0103,0.0108,0.0112,0.0114,0.0118,0.0124,0.0130,0.0136,0.0137,0.0133,0.0139,0.0156,0.0164,0.0162,0.0168,0.0181,0.0195,0.0211,0.0225,0.0237,0.0267,0.0315,0.0362,0.0408,0.0523,0.0707,0.0875,0.1027,0.1190,0.1362,0.1716,0.2252,0.3319,0.4915,0.5714,
    0.0082,0.0080,0.0075,0.0075,0.0078,0.0081,0.0084,0.0084,0.0082,0.0083,0.0085,0.0088,0.0092,0.0096,0.0099,0.0103,0.0108,0.0112,0.0115,0.0119,0.0125,0.0130,0.0136,0.0138,0.0136,0.0142,0.0156,0.0164,0.0166,0.0173,0.0184,0.0196,0.0210,0.0223,0.0235,0.0265,0.0313,0.0358,0.0400,0.0500,0.0656,0.0816,0.0979,0.1139,0.1296,0.1598,0.2044,0.2873,0.4085,0.4691,
    0.0077,0.0075,0.0072,0.0072,0.0076,0.0079,0.0082,0.0083,0.0081,0.0082,0.0086,0.0089,0.0093,0.0096,0.0099,0.0103,0.0107,0.0112,0.0116,0.0121,0.0126,0.0131,0.0137,0.0140,0.0142,0.0147,0.0156,0.0165,0.0175,0.0183,0.0190,0.0198,0.0208,0.0219,0.0232,0.0262,0.0310,0.0352,0.0386,0.0454,0.0555,0.0698,0.0882,0.1038,0.1165,0.1362,0.1628,0.1982,0.2425,0.2647,
    0.0069,0.0069,0.0069,0.0071,0.0074,0.0077,0.0080,0.0082,0.0081,0.0082,0.0085,0.0089,0.0093,0.0096,0.0099,0.0102,0.0106,0.0111,0.0116,0.0121,0.0125,0.0130,0.0136,0.0141,0.0145,0.0150,0.0156,0.0165,0.0177,0.0185,0.0189,0.0196,0.0205,0.0216,0.0230,0.0260,0.0306,0.0344,0.0373,0.0419,0.0481,0.0597,0.0769,0.0918,0.1045,0.1162,0.1271,0.1347,0.1390,0.1412,
    0.0060,0.0062,0.0067,0.0070,0.0071,0.0074,0.0078,0.0080,0.0081,0.0082,0.0084,0.0087,0.0091,0.0095,0.0098,0.0101,0.0105,0.0109,0.0115,0.0119,0.0123,0.0128,0.0134,0.0140,0.0146,0.0152,0.0156,0.0163,0.0171,0.0178,0.0183,0.0191,0.0201,0.0214,0.0230,0.0259,0.0300,0.0335,0.0362,0.0395,0.0432,0.0514,0.0639,0.0779,0.0934,0.0998,0.0974,0.0967,0.0980,0.0986,
    0.0053,0.0057,0.0063,0.0067,0.0068,0.0071,0.0076,0.0079,0.0079,0.0081,0.0083,0.0086,0.0089,0.0093,0.0096,0.0100,0.0103,0.0107,0.0113,0.0117,0.0121,0.0126,0.0132,0.0138,0.0145,0.0150,0.0155,0.0161,0.0168,0.0174,0.0180,0.0188,0.0200,0.0213,0.0229,0.0253,0.0286,0.0318,0.0349,0.0380,0.0410,0.0465,0.0546,0.0653,0.0786,0.0829,0.0784,0.0752,0.0734,0.0725,
    0.0050,0.0052,0.0058,0.0062,0.0065,0.0069,0.0074,0.0077,0.0077,0.0078,0.0081,0.0084,0.0088,0.0091,0.0094,0.0097,0.0102,0.0106,0.0109,0.0114,0.0120,0.0125,0.0131,0.0136,0.0141,0.0146,0.0152,0.0158,0.0166,0.0173,0.0180,0.0189,0.0201,0.0214,0.0227,0.0243,0.0263,0.0293,0.0333,0.0373,0.0413,0.0452,0.0490,0.0540,0.0600,0.0655,0.0703,0.0702,0.0651,0.0626,
    0.0048,0.0050,0.0055,0.0059,0.0062,0.0066,0.0071,0.0074,0.0074,0.0076,0.0079,0.0082,0.0085,0.0088,0.0092,0.0095,0.0100,0.0104,0.0107,0.0112,0.0119,0.0124,0.0128,0.0132,0.0136,0.0142,0.0148,0.0155,0.0163,0.0171,0.0179,0.0188,0.0199,0.0211,0.0225,0.0238,0.0252,0.0278,0.0316,0.0353,0.0390,0.0421,0.0447,0.0471,0.0492,0.0541,0.0619,0.0633,0.0582,0.0556,
    0.0047,0.0050,0.0054,0.0057,0.0059,0.0062,0.0067,0.0070,0.0070,0.0072,0.0077,0.0080,0.0082,0.0085,0.0090,0.0094,0.0098,0.0102,0.0106,0.0111,0.0117,0.0122,0.0125,0.0128,0.0131,0.0136,0.0143,0.0151,0.0157,0.0166,0.0177,0.0187,0.0195,0.0207,0.0223,0.0238,0.0253,0.0273,0.0299,0.0321,0.0341,0.0373,0.0417,0.0446,0.0460,0.0489,0.0534,0.0546,0.0525,0.0514,
    0.0047,0.0049,0.0053,0.0056,0.0057,0.0060,0.0064,0.0067,0.0068,0.0070,0.0075,0.0078,0.0080,0.0083,0.0087,0.0091,0.0096,0.0100,0.0103,0.0108,0.0115,0.0119,0.0123,0.0125,0.0127,0.0132,0.0140,0.0147,0.0153,0.0162,0.0174,0.0184,0.0192,0.0204,0.0221,0.0237,0.0251,0.0267,0.0285,0.0299,0.0311,0.0340,0.0387,0.0416,0.0429,0.0449,0.0476,0.0486,0.0477,0.0473,
    0.0045,0.0047,0.0051,0.0054,0.0057,0.0060,0.0063,0.0066,0.0067,0.0069,0.0072,0.0075,0.0079,0.0082,0.0085,0.0089,0.0094,0.0098,0.0100,0.0104,0.0111,0.0116,0.0120,0.0124,0.0127,0.0131,0.0138,0.0144,0.0149,0.0157,0.0169,0.0180,0.0191,0.0204,0.0219,0.0234,0.0248,0.0261,0.0274,0.0287,0.0300,0.0323,0.0357,0.0382,0.0399,0.0421,0.0447,0.0452,0.0438,0.0430,
    0.0044,0.0045,0.0048,0.0052,0.0056,0.0060,0.0062,0.0064,0.0065,0.0067,0.0070,0.0073,0.0077,0.0080,0.0083,0.0087,0.0092,0.0095,0.0097,0.0101,0.0107,0.0113,0.0117,0.0121,0.0125,0.0129,0.0136,0.0141,0.0146,0.0153,0.0164,0.0175,0.0186,0.0200,0.0215,0.0230,0.0243,0.0256,0.0269,0.0282,0.0296,0.0315,0.0339,0.0361,0.0380,0.0400,0.0418,0.0424,0.0416,0.0412,
    0.0041,0.0042,0.0045,0.0049,0.0054,0.0058,0.0061,0.0062,0.0062,0.0064,0.0068,0.0072,0.0075,0.0079,0.0082,0.0085,0.0089,0.0092,0.0094,0.0098,0.0104,0.0109,0.0114,0.0118,0.0122,0.0126,0.0133,0.0138,0.0142,0.0150,0.0160,0.0170,0.0179,0.0192,0.0209,0.0224,0.0237,0.0252,0.0268,0.0284,0.0300,0.0316,0.0332,0.0351,0.0373,0.0386,0.0392,0.0400,0.0411,0.0417,
    0.0041,0.0042,0.0044,0.0047,0.0052,0.0056,0.0058,0.0060,0.0061,0.0063,0.0067,0.0070,0.0074,0.0076,0.0079,0.0083,0.0087,0.0090,0.0092,0.0095,0.0101,0.0106,0.0111,0.0115,0.0119,0.0124,0.0130,0.0135,0.0139,0.0146,0.0156,0.0165,0.0174,0.0186,0.0202,0.0218,0.0232,0.0248,0.0265,0.0283,0.0300,0.0315,0.0328,0.0345,0.0366,0.0376,0.0376,0.0388,0.0413,0.0425,
    0.0044,0.0044,0.0045,0.0046,0.0049,0.0052,0.0055,0.0058,0.0060,0.0062,0.0066,0.0069,0.0072,0.0074,0.0075,0.0079,0.0084,0.0087,0.0090,0.0093,0.0097,0.0102,0.0106,0.0112,0.0118,0.0123,0.0128,0.0132,0.0136,0.0143,0.0152,0.0162,0.0171,0.0183,0.0196,0.0211,0.0229,0.0246,0.0261,0.0278,0.0298,0.0314,0.0327,0.0342,0.0359,0.0368,0.0370,0.0388,0.0421,0.0437,
    0.0046,0.0046,0.0045,0.0046,0.0048,0.0051,0.0054,0.0057,0.0059,0.0062,0.0066,0.0068,0.0071,0.0072,0.0073,0.0077,0.0082,0.0086,0.0088,0.0092,0.0096,0.0100,0.0104,0.0110,0.0117,0.0123,0.0127,0.0131,0.0135,0.0142,0.0150,0.0160,0.0170,0.0181,0.0192,0.0208,0.0227,0.0244,0.0259,0.0276,0.0297,0.0313,0.0326,0.0341,0.0356,0.0365,0.0368,0.0388,0.0424,0.0443]).reshape((32,50))

    epv = cv2.resize(epv, (106, 69))
    
    return epv

#############################
# xG Model from Environment #
#############################
def get_xg():
    xg_map = numpy.array([[ 0 , 0 , 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 0],
    [ 0 , 0 , 0 , 0 , 0 , 0  ,0,  0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0, 0],
    [ 0 , 0 , 0 , 0 , 0 ,0,  0,  0 , 0 , 0 , 0,  0,  0 , 0 , 0 , 0 , 0,  0 , 0,  0,  0 , 0 , 0 , 0 ,0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0,  0 , 0 , 0,  0 , 0 , 0  ,0 , 0 , 0 , 0 , 0,  0,  0,  0,  0,  0,  0,  1],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  0 , 0 , 0  ,0 , 0 , 0 , 0 , 0,  0,  0 , 0 , 0,  0,  0, 0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  0 , 0 , 1  ,0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0,  0,   0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  0 , 0 , 0  ,0 , 0 , 0 , 0 , 0,  0 , 0 , 0  ,0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 1,  2 , 6 , 1 , 0 , 0 , 0 , 1 , 0,  0 , 0 , 1  ,0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  1 , 1 , 2 , 0,  5 , 1 , 3 , 2 , 3 , 0 , 1 , 2,  1 , 0 , 1  ,0 , 0 , 1 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  4 , 3 , 1 , 0,  0 , 0 , 1 , 0 , 2 , 3 , 2 , 2,  0 , 2 , 0  ,0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 1 , 0 , 1 , 2 , 3 , 0,  0 , 0 , 0 , 0 , 4 , 0 ,10 , 0,  1 , 3 , 1  ,4 , 1 , 0 , 1],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 1 , 3,  0 , 1 , 3 , 0 , 3 , 6 ,12 ,11,  8 , 6 , 5  ,6 , 7 , 2 , 6],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 2,  3 , 7 ,12 ,10 ,14, 18 ,16 ,23, 17 ,19 ,22 ,15 , 6 , 4 ,  1],
    [ 0 , 0 , 0 , 0 , 0  ,0,  9 , 0 , 9 , 1,  2 , 5 , 1 , 2 , 5, 14 ,11 ,10,  1 , 4 , 7 , 2 , 5 , 2 ,  4],
    [ 0 , 0 , 0 , 0 , 0 , 1,  2 , 1 , 0 , 2,  0 , 1 , 0 , 1 , 0 , 3 , 4 , 4,  2 , 2 , 2 , 0 , 4 , 3 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  6 , 3 , 0 , 0,  3 , 0 , 1 , 1 , 0 , 1 , 5 , 0,  6 , 4 , 1 , 1 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 1 , 0,  1 , 0 , 2 , 3 , 1 , 1 , 0 , 0,  0 , 0 , 1 , 0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0,  2 , 2 , 0 , 0 , 5 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  1 , 1 , 1 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 1 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0  ,0,  0 , 0 , 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 , 1 , 0,  0 , 0 , 0 , 0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 , 0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 ,  0],
    [ 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0 , 0 , 0 ,  0]])

    xg_map = numpy.concatenate([numpy.zeros((25,25), dtype=int), xg_map], axis=1)/100
    xg_map = cv2.blur(xg_map, (7,7))
    return xg_map

#######################
# Pitch Control Model #
######################
# Extracted from: https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking
def calculate_pitch_control_at_target(target_position, attacking_players, defending_players, ball_start_pos, params, field_size = [106,68]):
    
    if target_position[0] < -field_size[0]/2 or target_position[0] > field_size[0]/2 or target_position[1] < -field_size[1]/2 or target_position[1] > field_size[1]/2:
        return 0., 1.

    # calculate ball travel time from start position to end position.
    if ball_start_pos is None or any(np.isnan(ball_start_pos)):  # assume that ball is already at location
        ball_travel_time = 0.0
    else:
        # ball travel time is distance to target position from current ball position divided assumed average ball speed
        ball_travel_time = np.linalg.norm(target_position - ball_start_pos) / params['average_ball_speed']

    # first get arrival time of 'nearest' attacking player (nearest also dependent on current velocity)
    tau_min_att = np.nanmin([p.simple_time_to_intercept(target_position) for p in attacking_players])
    tau_min_def = np.nanmin([p.simple_time_to_intercept(target_position) for p in defending_players])

    # check whether we actually need to solve equation 3
    if tau_min_att - max(ball_travel_time, tau_min_def) >= params['time_to_control_def']:
        # if defending team can arrive significantly before attacking team, no need to solve pitch control model
        return 0., 1.
    elif tau_min_def - max(ball_travel_time, tau_min_att) >= params['time_to_control_att']:
        # if attacking team can arrive significantly before defending team, no need to solve pitch control model
        return 1., 0.
    else:
        # solve pitch control model by integrating equation 3 in Spearman et al.
        # first remove any player that is far (in time) from the target location
        attacking_players = [p for p in attacking_players if
                             p.time_to_intercept - tau_min_att < params['time_to_control_att']]
        defending_players = [p for p in defending_players if
                             p.time_to_intercept - tau_min_def < params['time_to_control_def']]
        # set up integration arrays
        dT_array = np.arange(ball_travel_time - params['int_dt'], ball_travel_time + params['max_int_time'],
                             params['int_dt'])
        PPCFatt = np.zeros_like(dT_array)
        PPCFdef = np.zeros_like(dT_array)
        # integration equation 3 of Spearman 2018 until convergence or tolerance limit hit (see 'params')
        ptot = 0.0
        i = 1
        while 1 - ptot > params['model_converge_tol'] and i < dT_array.size:
            T = dT_array[i]
            for player in attacking_players:
                # calculate ball control probablity for 'player' in time interval T+dt
                dPPCFdT = (1 - PPCFatt[i - 1] - PPCFdef[i - 1]) * player.probability_intercept_ball(
                    T) * player.lambda_att
                # make sure it's greater than zero
                assert dPPCFdT >= 0, 'Invalid attacking player probability (calculate_pitch_control_at_target)'
                player.PPCF += dPPCFdT * params['int_dt']  # total contribution from individual player
                PPCFatt[
                    i] += player.PPCF  # add to sum over players in the attacking team (remembering array element is zero at the start of each integration iteration)
            for player in defending_players:
                # calculate ball control probablity for 'player' in time interval T+dt
                dPPCFdT = (1 - PPCFatt[i - 1] - PPCFdef[i - 1]) * player.probability_intercept_ball(
                    T) * player.lambda_def
                # make sure it's greater than zero
                assert dPPCFdT >= 0, 'Invalid defending player probability (calculate_pitch_control_at_target)'
                player.PPCF += dPPCFdT * params['int_dt']  # total contribution from individual player
                PPCFdef[i] += player.PPCF  # add to sum over players in the defending team
            ptot = PPCFdef[i] + PPCFatt[i]  # total pitch control probability
            i += 1
        if i >= dT_array.size:
            print("Integration failed to converge: %1.3f" % (ptot))
        return PPCFatt[i - 1], PPCFdef[i - 1]


def default_model_params(time_to_control_veto=3):

    # key parameters for the model, as described in Spearman 2018
    params = {}
    # model parameters
    params['max_player_accel'] = 7.  # maximum player acceleration m/s/s, not used in this implementation
    params['max_player_speed'] = 5.  # maximum player speed m/s
    params[
        'reaction_time'] = 0.7  # seconds, time taken for player to react and change trajectory. Roughly determined as vmax/amax
    params[
        'tti_sigma'] = 0.45  # Standard deviation of sigmoid function in Spearman 2018 ('s') that determines uncertainty in player arrival time
    params[
        'kappa_def'] = 1.  # kappa parameter in Spearman 2018 (=1.72 in the paper) that gives the advantage defending players to control ball, I have set to 1 so that home & away players have same ball control probability
    params['lambda_att'] = 4.3  # ball control parameter for attacking team
    params['lambda_def'] = 4.3 * params['kappa_def']  # ball control parameter for defending team
    params['lambda_gk'] = params[
                              'lambda_def'] * 3.0  # make goal keepers must quicker to control ball (because they can catch it)
    params['average_ball_speed'] = 15.  # average ball travel speed in m/s
    # numerical parameters for model evaluation
    params['int_dt'] = 0.04  # integration timestep (dt)
    params['max_int_time'] = 10  # upper limit on integral time
    params['model_converge_tol'] = 0.01  # assume convergence when PPCF>0.99 at a given location.
    # The following are 'short-cut' parameters. We do not need to calculated PPCF explicitly when a player has a sufficient head start.
    # A sufficient head start is when the a player arrives at the target location at least 'time_to_control' seconds before the next player
    params['time_to_control_att'] = time_to_control_veto * np.log(10) * (
                np.sqrt(3) * params['tti_sigma'] / np.pi + 1 / params['lambda_att'])
    params['time_to_control_def'] = time_to_control_veto * np.log(10) * (
                np.sqrt(3) * params['tti_sigma'] / np.pi + 1 / params['lambda_def'])
    return params


class Player(object):
    # player object holds position, velocity, time-to-intercept and pitch control contributions for each player
    def __init__(self, pid, pos, teamname, params, GKid):
        self.id = pid
        self.is_gk = self.id == GKid
        self.teamname = teamname
        self.playername = "%s_%s_" % (teamname, pid)
        self.vmax = params['max_player_speed']  # player max speed in m/s. Could be individualised
        self.reaction_time = params['reaction_time']  # player reaction time in 's'. Could be individualised
        self.tti_sigma = params['tti_sigma']  # standard deviation of sigmoid function (see Eq 4 in Spearman, 2018)
        self.lambda_att = params['lambda_att']  # standard deviation of sigmoid function (see Eq 4 in Spearman, 2018)
        self.lambda_def = params['lambda_gk'] if self.is_gk else params[
            'lambda_def']  # factor of 3 ensures that anything near the GK is likely to be claimed by the GK
        self.position = pos
        self.velocity = numpy.array([0,0])
        self.PPCF = 0.  # initialise this for later

    # def get_position(self, team):
    #     self.position = np.array([team[self.playername + 'x'], team[self.playername + 'y']])
    #     self.inframe = not np.any(np.isnan(self.position))
    #
    # def get_velocity(self, team):
    #     self.velocity = np.array([team[self.playername + 'vx'], team[self.playername + 'vy']])
    #     if np.any(np.isnan(self.velocity)):
    #         self.velocity = np.array([0., 0.])

    def simple_time_to_intercept(self, r_final):
        self.PPCF = 0.  # initialise this for later
        # Time to intercept assumes that the player continues moving at current velocity for 'reaction_time' seconds
        # and then runs at full speed to the target position.
        r_reaction = self.position + self.velocity * self.reaction_time
        self.time_to_intercept = self.reaction_time + np.linalg.norm(r_final - r_reaction) / self.vmax
        return self.time_to_intercept

    def probability_intercept_ball(self, T):
        # probability of a player arriving at target location at time 'T' given their expected time_to_intercept (time of arrival), as described in Spearman 2018
        f = 1 / (1. + np.exp(-np.pi / np.sqrt(3.0) / self.tti_sigma * (T - self.time_to_intercept)))
        return f


###############################
# Weight function equation (1)#
###############################
def weight_function(pc, epv, xg = 0):

    global wpc
    global wepv
    global wxg

    return wpc*pc*epv + wepv*epv + wxg*xg


def get_allowed_action_which_maximizes_epv(controlled_player_pos_normalized,
                                   controlled_player_dir_normalized,
                                   controlled_player_id,
                                   ball_pos_normalized,
                                   players_pos_left_normalized,
                                   players_pos_right_normalized,
                                   epv_map,
                                   xg_map,
                                   is_dribbling,
                                   is_sprinting):
    
    global field_size
  
    # Determine pass type
    pass_types, angles_pass_and_orientation, d , angles_pass_and_orientation_def, d_def = get_pass_type(controlled_player_pos_normalized, controlled_player_dir_normalized, players_pos_left_normalized, players_pos_right_normalized)
    # Player which environment will pass
    angles_pass_and_orientation[d==0] = 1000 # avoid current player
    angles_pass_and_orientation_def[0] = 1000 # avoid goalkeeper player
    player_to_pass_id = numpy.argmin(angles_pass_and_orientation)
    reciever_pos_normalized = players_pos_left_normalized[player_to_pass_id]
    min_angle = numpy.min(angles_pass_and_orientation)
    min_angle_def = numpy.min(angles_pass_and_orientation_def)
    min_dist = d[player_to_pass_id]
    min_dist_def = d_def[numpy.argmin(angles_pass_and_orientation_def)]

    epv_controlled_player = get_epv_from_normalized_point(controlled_player_pos_normalized, epv_map)
    epv_reciever_player = get_epv_from_normalized_point(reciever_pos_normalized, epv_map)
    
    deltam = 1
    params = default_model_params()
    attacking_players = []
    deffending_players = []
    for idx,p in enumerate(players_pos_left_normalized):
        attacking_players.append(Player(idx, p * field_size, "Home",params, 0))
    for idx,p in enumerate(players_pos_right_normalized):
        deffending_players.append(Player(idx, p * field_size, "Away",params, 0))

    orientations = numpy.array([[deltam, 0], [deltam, deltam], [0, deltam], [-deltam, deltam], [-deltam, 0], [-deltam, -deltam], [0, -deltam], [deltam, -deltam]]).reshape((-1,2))
    orientation_id = [5, 4, 3, 2, 1, 8, 7, 6]
    best_weight = -1
    best_orientation = -1
    for idx, orientation in enumerate(orientations):
        # Compute pitch control for each posible direction
        target_1 = controlled_player_pos_normalized * field_size + orientation
        target_5 = controlled_player_pos_normalized * field_size + orientation*10

        # Compute epv for each posible direction
        ppcf_att, ppcf_def = calculate_pitch_control_at_target(target_1, attacking_players, deffending_players, ball_pos_normalized.flatten(), params)
        ppcf_att_5, ppcf_def_5 = calculate_pitch_control_at_target(target_5, attacking_players, deffending_players, ball_pos_normalized.flatten(), params)
        if ppcf_att_5 == 0:
            ppcf_att = 0
        epv_att = get_epv_from_normalized_point(target_5/field_size, epv_map)
        xg_att = get_xg_from_normalized_point(target_5/field_size, xg_map)

        # weight each option
        weight = weight_function(ppcf_att, epv_att, xg_att)
        if weight > best_weight:
            best_weight = weight
            best_orientation = orientation_id[idx]
            
    # Weight candidate player to pass
    pc_controlled,_ = calculate_pitch_control_at_target(controlled_player_pos_normalized * field_size, attacking_players, deffending_players, ball_pos_normalized.flatten(), params)
    pc_receiver, _ = calculate_pitch_control_at_target(reciever_pos_normalized * field_size, attacking_players, deffending_players, ball_pos_normalized.flatten(), params)
    epv_controlled= get_epv_from_normalized_point(controlled_player_pos_normalized, epv_map)
    epv_receiver= get_epv_from_normalized_point(reciever_pos_normalized, epv_map)
    xg_controlled= get_xg_from_normalized_point(controlled_player_pos_normalized, xg_map)
    xg_receiver= get_xg_from_normalized_point(reciever_pos_normalized, xg_map)
    weight_controlled = weight_function(pc_controlled,epv_controlled, xg_controlled)
    weight_receiver = weight_function(pc_receiver,epv_receiver, xg_receiver)
    
    # If there is a player which improves the weight it pass him
    if player_to_pass_id!= 0 and (weight_receiver > weight_controlled) and \
        not offside(controlled_player_pos_normalized, reciever_pos_normalized, players_pos_right_normalized) and \
        (min_angle < min_angle_def and min_dist<min_dist_def):
        if is_sprinting:
            return 15
        pass_type = pass_types[player_to_pass_id]
        return pass_type

    else:
        if not is_sprinting:
            return 13

        return best_orientation

########
# Agent#
########
def get_action(players_pos_left, players_dirs_left, ball_pos, ball_dir, controlled_player_id, controlled_team_id, player_pos_right, players_dirs_right, is_dribbling, is_sprinting, previous_action):

    global distance_shot_threshold
    
    ######################
    # Environment Manager# 
    ######################    
    
    steps = 5
    if controlled_team_id == 1:
        players_pos_left = players_pos_left + steps*players_dirs_left
        player_pos_right = player_pos_right + steps*players_dirs_right
        ball_pos = ball_pos + steps*ball_dir
    else:
        ball_pos = ball_pos + steps*ball_dir
        
    
    players_pos_left_normalized = to_normalized_space_for_players(players_pos_left) 
    players_pos_right_normalized = to_normalized_space_for_players(player_pos_right) 
    ball_pos_normalized = to_normalized_space_for_players(ball_pos.reshape((-1,2))) 
 
    players_pos_left_normalized[:,0] -= 0.5
    players_pos_left_normalized[:,1] = 1 - players_pos_left_normalized[:,1] - 0.5
    players_pos_right_normalized[:,0] -= 0.5
    players_pos_right_normalized[:,1] = 1 - players_pos_right_normalized[:,1] - 0.5
    ball_pos_normalized[:,0] -= 0.5
    ball_pos_normalized[:,1] = 1 - ball_pos_normalized[:,1] - 0.5
    

    players_dirs_left[:, 1] = -players_dirs_left[:, 1] 
    players_dirs_left_normalized = players_dirs_left/numpy.linalg.norm(players_dirs_left, axis=1).reshape((-1,1))


    controlled_player_pos = players_pos_left[controlled_player_id]
    controlled_player_pos_normalized = players_pos_left_normalized[controlled_player_id]
    controlled_player_dir_normalized = players_dirs_left_normalized[controlled_player_id]
    
    epv_map = get_epv()  
    xg_map = get_xg()

    ######################
    # Decision Module    # 
    ###################### 
    
    # Attacking Mode
    if controlled_team_id == 1:

        # Shot decision
        controlled_player_xg = get_xg_from_normalized_point(controlled_player_pos_normalized, xg_map)
        distance_between_gk_and_controlled_player = numpy.linalg.norm((controlled_player_pos_normalized - players_pos_right_normalized[0])*numpy.array([106,68]))
        
        # Optimizes direction of the shoot
        if distance_shot_threshold < distance_between_gk_and_controlled_player < (distance_shot_threshold + 6):
            if controlled_player_pos_normalized[1] < 0:
                return 4
            else:
                return 6
        if distance_between_gk_and_controlled_player < distance_shot_threshold:
#         if controlled_player_xg > 0.04 or distance_between_gk_and_controlled_player < 14:
            if is_sprinting:
                return 15
            print("Shot")
            return 12

        # If Shot was not performed, select best action to carry out
        action = get_allowed_action_which_maximizes_epv(controlled_player_pos_normalized,
                                              controlled_player_dir_normalized,
                                              controlled_player_id,
                                              ball_pos_normalized,
                                              players_pos_left_normalized,
                                              players_pos_right_normalized,
                                              epv_map,
                                              xg_map,
                                              is_dribbling,
                                              is_sprinting)
        if 0<action<=9:
            if action!=previous_action and previous_action!=14:
                return 14
        
        return action    
    # Defending Mode
    else:
        if not is_sprinting:
            return 13
        # Select orientation based on the ball position
        orientation = get_orientation_to_pass(controlled_player_pos_normalized,
                                              ball_pos_normalized)
        return orientation

    
previous_pass_step = 100000
previous_action = -1

@human_readable_agent
def agent(obs):
    
    global previous_pass_step
    global previous_action
    
    # Getting data   
    players_pos_left = numpy.array(obs['left_team']).reshape((-1,2))
    players_pos_right = numpy.array(obs['right_team']).reshape((-1,2))
    players_dirs_left = numpy.array(obs['left_team_direction']).reshape((-1,2))
    players_dirs_right = numpy.array(obs['right_team_direction']).reshape((-1,2))
    ball_pos = numpy.array([obs['ball'][0], obs['ball'][1]])
    ball_dir = numpy.array([obs['ball_direction'][0], obs['ball_direction'][1]])
    
    controlled_player_id = obs['active']
    controlled_team_id = obs['ball_owned_team'] + 1
    steps_left = obs['steps_left']
    is_dribbling = False
    is_sprinting = False
    if Action.Dribble in obs['sticky_actions']:
        is_dribbling = True
    if Action.Sprint in obs['sticky_actions']:
        is_sprinting = True
    game_mode = obs['game_mode']
    
    # Shot for Penalty and GoalKick
    if game_mode==6 or game_mode==2:
        return Action.Shots
    
    ########
    # Agent#
    ########
    action = get_action(players_pos_left, 
                        players_dirs_left, 
                        ball_pos, 
                        ball_dir,
                        controlled_player_id,
                        controlled_team_id, 
                        players_pos_right,
                        players_dirs_right,
                        is_dribbling, 
                        is_sprinting,
                        previous_action)  
    
    previous_action = action
    
    # Traslate action int into action enum type
    if action == 0:
        return Action.Idle
    if action == 1:
        return Action.Left
    if action == 2:
        return Action.TopLeft
    if action == 3:
        return Action.Top
    if action == 4:
        return Action.TopRight
    if action == 5:
        return Action.Right
    if action == 6:
        return Action.BottomRight
    if action == 7:
        return Action.Bottom
    if action == 8:
        return Action.BottomLeft
    if action == 9:
        # This fix bug for consecutive passes 
        if previous_pass_step - steps_left < 10:
            return Action.Idle
        else:
            previous_pass_step = steps_left
            return Action.LongPass
    if action == 10:
        # This fix bug for consecutive passes 
        if previous_pass_step - steps_left < 10:
            return Action.Idle
        else:
            previous_pass_step = steps_left
            return Action.HighPass
    if action == 11:
        # This fix bug for consecutive passes 
        if previous_pass_step - steps_left < 10:
            return Action.Idle
        else:
            previous_pass_step = steps_left
            return Action.ShortPass
    if action == 12:
        return Action.Shot
    if action == 13:
        return Action.Sprint
    if action == 14:
        return Action.ReleaseDirection
    if action == 15:
        return Action.ReleaseSprint
    if action == 16:
        return Action.Slide
    if action == 17:
        return Action.Dribble
    if action == 18:
        return Action.ReleaseDribble
        
    return Action.Right


In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/submission.py", "run_right"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)